# PFS: classical ML models

In [ ]:
#Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
#Importing libraries for preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
#Importing libraries for models
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb
from lightgbm import LGBMRegressor
#Importing libraries for evaluation
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error
#Importing library for interpretability
import shap
#For reproducibility
SEED = 42
tf.keras.utils.set_random_seed(SEED)


In [ ]:
# load the data
data = pd.read_csv('dataset_b.csv', encoding='latin-1', sep=',') # request the dataset to the author

#data.head()

In [ ]:
# target column : "pfsduree_immunoth", continuous variable
# relevant columns
relevant_columns = ['age', 'dcr', 'dnlr', 'histology', 'immuno_line', 'iorr', 
                    'ldhpre', 'leucotpre', 'nb_meta_beforeimmuno', 'neuttpre', 
                     'ps_befimmuno', 'sex', 'smoking_history', 'pfsduree_immunoth']

data = data[relevant_columns]
data = data.dropna(axis=0)
data['dcr'] = data['dcr'].astype(int)
data['age'] = data['age'].astype(int)
data['iorr'] = data['iorr'].astype(int)
data['ps_befimmuno'] = data['ps_befimmuno'].astype(int)

In [ ]:
# "encode" the categorical variables
data['histology'] = data['histology'].str.lower()
data['sex'] = data['sex'].str.lower()
data['smoking_history'] = data['smoking_history'].str.lower()

In [ ]:
#to randomize the data
data = data.sample(frac=1, random_state=SEED)

# one-hot encoding
one_hot_data = pd.get_dummies(data, columns=['histology', 'sex', 'smoking_history'])

one_hot_data = one_hot_data.rename(columns={
    'histology_Adenocarcinoma': 'histology_adenocarcinoma',
    'histology_Squamous': 'histology_squamous',
    'histology_Nsclc_other': 'histology_nsclc_other',
    'histology_Large_cells': 'histology_large_cells',
    'sex_Male': 'sex_male',
    'sex_Female': 'sex_female',
    'smoking_history_Non_smoker': 'smoking_history_non_smoker',
    'smoking_history_Former': 'smoking_history_former',
    'smoking_history_Current': 'smoking_history_current',
    'smoking_history_Unk': 'smoking_history_unk'
})


In [ ]:
# replace boolean values with 0 and 1
for col in ['histology_adenocarcinoma','histology_squamous','histology_nsclc other',
    'histology_large cells','sex_male','sex_female','smoking_history_non smoker','smoking_history_former','smoking_history_current',
     'smoking_history_unk']:
    one_hot_data[col] = one_hot_data[col].replace({False: 0, True: 1})

In [ ]:
# split the data into features and target
X = one_hot_data[one_hot_data.columns.difference(['pfsduree_immunoth'])]
y = data['pfsduree_immunoth']

# First division: training+val vs. test (80% vs. 20%)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42  
)

# Second division: training vs. validation (75% vs. 25% of the 80%)
# This results in 60% training, 20% validation, 20% test   
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42
)

In [ ]:
#This ensures that all numerical features contribute equally
numerical_features = ['age', 'dcr', 'dnlr', 'ldhpre', 'leucotpre', 
                      'nb_meta_beforeimmuno', 'neuttpre', 'ps_befimmuno']

binary_features = [col for col in X.columns if col not in numerical_features]

scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_val_scaled = X_val.copy() 
X_test_scaled = X_test.copy()
X_scaled = X.copy()
X_train_val_scaled = X_temp.copy()

X_scaled[numerical_features] = scaler.fit_transform(X_scaled[numerical_features])
X_train_scaled[numerical_features] = scaler.fit_transform(X_train_scaled[numerical_features])
X_val_scaled[numerical_features] = scaler.transform(X_val_scaled[numerical_features])
X_test_scaled[numerical_features] = scaler.transform(X_test_scaled[numerical_features])
X_train_val_scaled[numerical_features] = scaler.fit_transform(X_train_val_scaled[numerical_features])

In [ ]:
def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    denom = (np.abs(y_true) + np.abs(y_pred)) / 2
    denom = np.where(denom == 0, 1e-8, denom)  # to avoid division by zero
    return np.mean(np.abs(y_true - y_pred) / denom) * 100


def evaluate_regression(y_true, y_pred):
    """
    Evaluate a regression model with MSE, MAE, MAPE and sMAPE.

    Parametri:
        y_true: array-like, true values
        y_pred: array-like, predicted values
    """

    # Metric calculations
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    smape = symmetric_mean_absolute_percentage_error(y_true, y_pred)

    # Print results
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
    print(f"Symmetric Mean Absolute Percentage Error (sMAPE): {smape:.2f}%")

    # Plot: Predicted vs True
    plt.figure(figsize=(6,6))
    sns.scatterplot(x=y_true, y=y_pred, alpha=0.6)
    plt.plot([y_true.min(), y_true.max()],
             [y_true.min(), y_true.max()],
             'r--', label='Perfect Prediction')
    plt.xlabel("True Values")
    plt.ylabel("Predicted Values")
    plt.title(f"Predicted vs True ")
    plt.legend()
    plt.show()


    return {"MSE": mse, "MAE": mae, "MAPE": mape, "sMAPE": smape}


# Linear Regression

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred, epsilon=1e-8):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    denom = np.maximum(np.abs(y_true), epsilon)  # to avoid division by zero
    return np.mean(np.abs((y_true - y_pred) / denom)) * 100

mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)


scoring = {
    'neg_mse': 'neg_mean_squared_error',
    'neg_mae': 'neg_mean_absolute_error',
    'neg_mape': mape_scorer
}

param_grid_linear = {
    'fit_intercept': [True, False]
}

grid = GridSearchCV(
    LinearRegression(),
    param_grid=param_grid_linear,
    scoring=scoring,
    refit='neg_mae',
    cv=5,
    verbose=1,
    n_jobs=1
)

grid.fit(X_train_val_scaled, y_temp)
print("Best Linear params:", grid.best_params_)
print("Best MAE:", -grid.best_score_)


In [ ]:
lr_model_best = grid.best_estimator_

lr_model_best.fit(X_train_val_scaled, y_temp)

y_pred_lr_best = lr_model_best.predict(X_test_scaled)

print("\nTest Set Evaluation:")
evaluate_regression(y_test, y_pred_lr_best)

In [ ]:
X_train_val_scaled_df = pd.DataFrame(X_train_val_scaled, columns=X.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X.columns)

In [ ]:
explainer = shap.Explainer(lr_model_best, X_train_val_scaled_df)

shap_values = explainer(X_test_scaled_df)

print("SHAP summary plot:")
shap.plots.beeswarm(shap_values, max_display=len(binary_features) + len(numerical_features))

# Ridge

In [ ]:
param_grid_ridge = {
    'alpha': [0.001, 0.01, 0.1, 1, 10, 100],
    'fit_intercept': [True, False],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sag', 'lbfgs']
}

grid = GridSearchCV(
    Ridge(max_iter=10000, random_state=SEED),
    param_grid=param_grid_ridge,
    scoring=scoring,
    refit='neg_mae',
    cv=5,
    verbose=1,
    n_jobs=1
)

grid.fit(X_train_val_scaled, y_temp)
print("Best Ridge params:", grid.best_params_)
print("Best MAE:", -grid.best_score_)


In [ ]:
ridge_model_best = grid.best_estimator_

ridge_model_best.fit(X_train_val_scaled, y_temp)

y_pred_lr_best = ridge_model_best.predict(X_test_scaled)

print("\nTest Set Evaluation:")
evaluate_regression(y_test, y_pred_lr_best)

In [ ]:
explainer = shap.Explainer(ridge_model_best, X_train_val_scaled_df)

shap_values = explainer(X_test_scaled_df)

print("SHAP summary plot:")
shap.plots.beeswarm(shap_values, max_display=len(binary_features) + len(numerical_features))

# Lasso

In [ ]:
param_grid_lasso = {
    'alpha': [0.001, 0.01, 0.1, 1, 10],
    'fit_intercept': [True, False],
    'max_iter': [1000, 5000, 10000]
}

grid = GridSearchCV(
    Lasso(random_state=SEED),
    param_grid=param_grid_lasso,
    scoring=scoring,
    refit='neg_mae',
    cv=5,
    verbose=1,
    n_jobs=1
)

grid.fit(X_train_val_scaled, y_temp)
print("Best Lasso params:", grid.best_params_)
print("Best MAE:", -grid.best_score_)


In [ ]:
lasso_model_best = grid.best_estimator_

lasso_model_best.fit(X_train_val_scaled, y_temp)

y_pred_lr_best = lasso_model_best.predict(X_test_scaled)

print("\nTest Set Evaluation:")
evaluate_regression(y_test, y_pred_lr_best)

In [ ]:
explainer = shap.Explainer(lasso_model_best, X_train_val_scaled_df)

shap_values = explainer(X_test_scaled_df)

print("SHAP summary plot:")
shap.plots.beeswarm(shap_values, max_display=len(binary_features) + len(numerical_features))

# ElasticNet

In [ ]:
param_grid_elastic = {
    'alpha': [0.001, 0.01, 0.1, 1, 10],
    'l1_ratio': [0.1, 0.5, 0.7, 0.9, 1.0],
    'fit_intercept': [True, False],
    'max_iter': [1000, 5000, 10000]
}

grid = GridSearchCV(
    ElasticNet(random_state=SEED),
    param_grid=param_grid_elastic,
    scoring=scoring,
    refit='neg_mae',
    cv=5,
    verbose=1,
    n_jobs=1
)

grid.fit(X_train_val_scaled, y_temp)
print("Best ElasticNet params:", grid.best_params_)
print("Best MAE:", -grid.best_score_)


In [ ]:
elastic_model_best = grid.best_estimator_

elastic_model_best.fit(X_train_val_scaled, y_temp)

y_pred_lr_best = elastic_model_best.predict(X_test_scaled)

print("\nTest Set Evaluation:")
evaluate_regression(y_test, y_pred_lr_best)

In [ ]:
explainer = shap.Explainer(elastic_model_best, X_train_val_scaled_df)

shap_values = explainer(X_test_scaled_df)

print("SHAP summary plot:")
shap.plots.beeswarm(shap_values, max_display=len(binary_features + numerical_features))


# Decision Tree Regressor

In [ ]:
param_grid_dt = {
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid = GridSearchCV(
    DecisionTreeRegressor(random_state=SEED),
    param_grid=param_grid_dt,
    scoring=scoring,
    refit='neg_mae',
    cv=5,
    verbose=1,
    n_jobs=1
)
grid.fit(X_temp, y_temp)
print("Best Decision Tree params:", grid.best_params_)
print("Best MAE:", -grid.best_score_)

In [ ]:
dt_model_best = grid.best_estimator_

dt_model_best.fit(X_temp, y_temp)

y_pred_lr_best = dt_model_best.predict(X_test)

print("\nTest Set Evaluation:")
evaluate_regression(y_test, y_pred_lr_best)

In [ ]:
X_temp_df = pd.DataFrame(X_temp, columns=X_temp.columns)
X_test_df = pd.DataFrame(X_test, columns=X_test.columns)

In [ ]:
explainer = shap.Explainer(dt_model_best, X_temp_df)

shap_values = explainer(X_test_df)

print("SHAP summary plot:")
shap.plots.beeswarm(shap_values, max_display=len(binary_features + numerical_features))


# Random Forest

In [ ]:
param_grid_rf = {
    'n_estimators': [100, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['auto', 'sqrt']
}

grid = GridSearchCV(
    RandomForestRegressor(random_state=SEED),
    param_grid=param_grid_rf,
    scoring=scoring,
    refit='neg_mae',
    cv=5,
    verbose=1,
    n_jobs=1
)
grid.fit(X_temp, y_temp)
print("Best Random Forest params:", grid.best_params_)
print("Best MAE:", -grid.best_score_)


In [ ]:
rf_model_best = grid.best_estimator_

rf_model_best.fit(X_temp, y_temp)

y_pred_lr_best = rf_model_best.predict(X_test)

print("\nTest Set Evaluation:")
evaluate_regression(y_test, y_pred_lr_best)

In [ ]:
explainer = shap.Explainer(rf_model_best, X_temp_df)

shap_values = explainer(X_test_df)

print("SHAP summary plot:")
shap.plots.beeswarm(shap_values, max_display=len(binary_features + numerical_features))


# Gradient Boosting Regressor

In [ ]:
param_grid_gb = {
    'n_estimators': [100, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

grid = GridSearchCV(
    GradientBoostingRegressor(random_state=SEED),
    param_grid=param_grid_gb,
    scoring=scoring,
    refit='neg_mae',
    cv=5,
    verbose=1,
    n_jobs=1
)
grid.fit(X_temp, y_temp)
print("Best Gradient Boosting params:", grid.best_params_)
print("Best MAE:", -grid.best_score_)


In [ ]:
gb_model_best = grid.best_estimator_

gb_model_best.fit(X_temp, y_temp)

y_pred_lr_best = gb_model_best.predict(X_test)

print("\nTest Set Evaluation:")
evaluate_regression(y_test, y_pred_lr_best)

In [ ]:
explainer = shap.Explainer(gb_model_best, X_temp_df)

shap_values = explainer(X_test_df)

print("SHAP summary plot:")
shap.plots.beeswarm(shap_values, max_display=len(binary_features + numerical_features))


# K-Neighbors Regressor

In [ ]:
param_grid_knn = {
    'n_neighbors': [3, 5, 7, 10],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]  # 1 = Manhattan, 2 = Euclidean
}

grid = GridSearchCV(
    KNeighborsRegressor(),
    param_grid=param_grid_knn,
    scoring=scoring,
    refit='neg_mae',
    cv=5,
    verbose=1,
    n_jobs=1
)
grid.fit(X_train_val_scaled, y_temp)
print("Best KNN params:", grid.best_params_)
print("Best MAE:", -grid.best_score_)

In [ ]:
knn_model_best = grid.best_estimator_

knn_model_best.fit(X_train_val_scaled, y_temp)

y_pred_lr_best = knn_model_best.predict(X_test_scaled)

print("\nTest Set Evaluation:")
evaluate_regression(y_test, y_pred_lr_best)

In [ ]:
X_sample = X_train_val_scaled_df.sample(100, random_state=SEED)

explainer = shap.KernelExplainer(knn_model_best.predict, X_sample)

X_test_sample = X_test_scaled_df.sample(50, random_state=SEED)
shap_values = explainer.shap_values(X_test_sample)

print("SHAP summary plot:")
shap.summary_plot(shap_values, X_test_sample, feature_names=X_test_sample.columns)

# Support Vector Regressor (SVR)

In [ ]:
param_grid_svr = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'epsilon': [0.01, 0.1, 1.0],
    'gamma': ['scale', 'auto']
}

grid = GridSearchCV(
    SVR(),
    param_grid=param_grid_svr,
    scoring=scoring,
    refit='neg_mae',
    cv=5,
    verbose=1,
    n_jobs=1
)
grid.fit(X_train_val_scaled, y_temp)
print("Best SVR params:", grid.best_params_)
print("Best MAE:", -grid.best_score_)

In [ ]:
svr_model_best = grid.best_estimator_

svr_model_best.fit(X_train_val_scaled, y_temp)

y_pred_lr_best = svr_model_best.predict(X_test_scaled)

print("\nTest Set Evaluation:")
evaluate_regression(y_test, y_pred_lr_best)

In [ ]:
explainer = shap.Explainer(svr_model_best, X_train_val_scaled_df)

shap_values = explainer(X_test_scaled_df)

print("SHAP summary plot:")
shap.plots.beeswarm(shap_values, max_display=len(binary_features + numerical_features))


# XGBoost Regressor

In [ ]:
param_grid_xgb = {
    'n_estimators': [100, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.7, 1.0]
}

grid = GridSearchCV(
    xgb.XGBRegressor(random_state=SEED, verbosity=0),
    param_grid=param_grid_xgb,
    scoring=scoring,
    refit='neg_mae',
    cv=5,
    verbose=1,
    n_jobs=1
)
grid.fit(X_temp, y_temp)
print("Best XGB params:", grid.best_params_)
print("Best MAE:", -grid.best_score_)


In [ ]:
xgb_model_best = grid.best_estimator_

xgb_model_best.fit(X_temp, y_temp)

y_pred_lr_best = xgb_model_best.predict(X_test)

print("\nTest Set Evaluation:")
evaluate_regression(y_test, y_pred_lr_best)

In [ ]:
explainer = shap.Explainer(xgb_model_best, X_temp_df)

shap_values = explainer(X_test_df)

print("SHAP summary plot:")
shap.plots.beeswarm(shap_values, max_display=len(binary_features + numerical_features))


# LightGBM Regressor

In [ ]:
param_grid_lgb = {
    'n_estimators': [100, 300],
    'max_depth': [-1, 5, 10],
    'learning_rate': [0.01, 0.1],
    'num_leaves': [31, 50]
}

grid = GridSearchCV(
    LGBMRegressor(random_state=SEED),
    param_grid=param_grid_lgb,
    scoring=scoring,
    refit='neg_mae',
    cv=5,
    verbose=1,
    n_jobs=1
)
grid.fit(X_temp, y_temp)
print("Best LGBM params:", grid.best_params_)
print("Best MAE:", -grid.best_score_)

In [ ]:
lgb_model_best = grid.best_estimator_

lgb_model_best.fit(X_temp, y_temp)

y_pred_lr_best = lgb_model_best.predict(X_test)

print("\nTest Set Evaluation:")
evaluate_regression(y_test, y_pred_lr_best)

In [ ]:
explainer = shap.Explainer(lgb_model_best, X_temp_df)

shap_values = explainer(X_test_df)

print("SHAP summary plot:")
shap.plots.beeswarm(shap_values, max_display=len(binary_features + numerical_features))